In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [58]:
# Import Data and methods
from scripts.proj1_helpers import *
from scripts.implementations import *
y_train_load, x_train_load, ids_train = load_csv_data('./data/train.csv', sub_sample=False)
_, x_test_load, ids_test = load_csv_data('./data/test.csv', sub_sample=False)

# Add 1 dimension FIXME not understood
y_train = np.expand_dims(y_train_load, axis=1)

# Logistic Regression Baseline

In [78]:
# build sampled x and y.
seed = 1
#y_train_samp, x_train_samp = sample_data(y_train, x_train, seed, size_samples=y_train_load.shape[0])

# standardize both sets
remove_aberrant(x_train_load)
remove_aberrant(x_test_load)
x_train, mean_x, std_x = standardize(x_train_load)
x_test, mean_test, std_test = standardize(x_test_load)

In [79]:
tx_train = np.c_[np.ones((y_train.shape[0], 1)), x_train]
tx_test = np.c_[np.ones((x_test.shape[0], 1)), x_test]

In [102]:
# logistic regression
initial_w = np.zeros((tx_train.shape[1], 1))
lambda_ = 400
gamma = 0.001
assert(lambda_ * gamma <= 1/2)
max_iters = 501

weights, loss = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iters, gamma)
#weights, loss = least_squares(y_train, tx_train)
print(loss)
# Sunday Morning todo:
# Problem exp overflow -> sigmoid is 1 -> log(0) -> nan
# Find reason for exp overflow? -> weights are not 0 anymore !!! -> huge number since 200'000 * weights
# this implies that:
# The accuracy I get is after a single update !!

Current iteration=0, loss=173286.79514000562
weights size:71883.92327298716
[[ -2.03666000e+02]
 [  2.60878135e+00]
 [ -8.33941282e+01]
 [ -3.33532744e+00]
 [  4.56866480e+01]
 [  4.39684422e+01]
 [  4.24177513e+01]
 [ -3.94195876e+01]
 [  2.90586227e+00]
 [ -3.62771829e+00]
 [  3.63630064e+01]
 [ -4.63680733e+01]
 [  6.44869325e+01]
 [  4.12599488e+01]
 [  5.58221553e+01]
 [ -2.23834212e-01]
 [ -1.04472587e+00]
 [ -7.58118729e+00]
 [  3.59803838e-01]
 [  9.78971891e-01]
 [  5.33114037e+00]
 [  1.77390454e+00]
 [  3.21590638e+01]
 [  3.16913111e+01]
 [  2.07761451e+01]
 [  5.84093585e-02]
 [ -4.19142831e-02]
 [ -3.02851427e+00]
 [  1.61418076e-01]
 [ -8.42878944e-01]
 [  3.18684808e+01]]


/home/yann/Documents/higgs-boson/scripts/implementations.py:162: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))
/home/yann/Documents/higgs-boson/scripts/implementations.py:169: RuntimeWarning: divide by zero encountered in log
  #return np.squeeze(-(y.T @ np.log(sigmoid(tx @ w)) + (1 - y).T @ np.log(1 - sigmoid(tx @ w))))


Current iteration=100, loss=nan
weights size:117828.69615423374
[[ -2.80218847e+02]
 [ -3.94187179e+00]
 [ -1.31888276e+02]
 [ -8.05088438e+00]
 [  4.66966353e+01]
 [  2.42279026e+01]
 [  2.68033366e+01]
 [ -1.82949255e+01]
 [  3.88467637e+01]
 [ -1.04524734e+01]
 [  2.42369947e+01]
 [ -6.20378567e+01]
 [  7.89169038e+01]
 [  3.24343943e+01]
 [  5.77366607e+01]
 [ -5.65556804e-01]
 [ -1.34242238e+00]
 [ -5.69776034e+00]
 [  1.25560373e-01]
 [  6.99437046e-01]
 [ -1.02555973e+01]
 [  1.36528493e+00]
 [  1.91147788e+01]
 [  1.82525486e+01]
 [  1.01943984e+01]
 [  1.15553309e-02]
 [  2.75043684e-01]
 [ -1.08915226e+01]
 [  3.66262437e-01]
 [ -7.98379421e-01]
 [  1.66921372e+01]]
Current iteration=200, loss=nan
weights size:117828.69615423374
[[ -2.80218847e+02]
 [ -3.94187179e+00]
 [ -1.31888276e+02]
 [ -8.05088438e+00]
 [  4.66966353e+01]
 [  2.42279026e+01]
 [  2.68033366e+01]
 [ -1.82949255e+01]
 [  3.88467637e+01]
 [ -1.04524734e+01]
 [  2.42369947e+01]
 [ -6.20378567e+01]
 [  7.89169

In [96]:
y_prediction = predict_labels(weights, tx_train)
print("Training set accuracy = "+str(compute_accuracy(y_train, y_prediction)))

Training set accuracy = 0.711864


## Helper methods

In [55]:
def compute_accuracy(y, y_prediction):
    correct = 0.0
    for i in range(len(y)):
        if y[i] == y_prediction[i]:
            correct += 1
    accuracy = correct/len(y)
    return accuracy

# replace -999 value by the mean of the corresponding feature without taking the aberrant values into account
def remove_aberrant(data):
    for i in range(data.shape[1]):
        data[:, i][data[:, i] == -999.0] = np.mean(data[:, i][data[:, i] != -999.0])


In [97]:
def cross_validation(y, x, k_fold, seed=1):
    """
    split the dataset in k equal subparts. Then train the model on k-1 parts and test it on the last remaining part.
    this process is repeatead k times with a different test part each time
    """
    assert(k_fold > 1)

    # set seed
    np.random.seed(seed)
    np.random.shuffle(x)
    # resetting the seed allows for an identical shuffling between y and x
    np.random.seed(seed)
    np.random.shuffle(y)
    
    loss_train = []
    loss_test = []
    acc_train, acc_test = [],[]
    
    total_length = len(y)
    split_length = int(total_length / k_fold)
    
    for i in range(k_fold):
        start_index = i * split_length
        if i == k_fold - 1:
            end_index = total_length
        else: #this case is to ensure that all elements are present, in case of unbalanced split
            end_index = (i + 1) * split_length
        
        tx_train = x[np.r_[0 : start_index, end_index: total_length]]
        y_train = y[np.r_[0 : start_index, end_index: total_length]]

        tx_test = x[start_index : end_index]
        y_test = y[start_index : end_index]

        # logistic regression
        initial_w = np.zeros((tx_train.shape[1], 1))
        lambda_ = 400
        gamma = 0.001
        assert(lambda_ * gamma <= 1/2)
        max_iters = 501

        weights, loss = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iters, gamma)
        y_train_pred = predict_labels(weights, tx_train)
        y_test_pred = predict_labels(weights, tx_test)
        
        loss_train.append(loss)
        loss_test.append(calculate_loss_logistic_regression(y_test, tx_test, weights))
        acc_train.append(compute_accuracy(y_train, y_train_pred))
        acc_test.append(compute_accuracy(y_test, y_test_pred))
        
    return np.mean(loss_train), np.mean(loss_test), np.mean(acc_train), np.mean(acc_test)

In [98]:
cross_validation(y_train, tx_train, 4, seed)

Current iteration=0, loss=129965.09635499373
weights size:40448.949896818514
[[ -1.52314000e+02]
 [  1.77095549e+00]
 [ -6.29424554e+01]
 [ -2.76646995e+00]
 [  3.44796443e+01]
 [  3.29896469e+01]
 [  3.19669766e+01]
 [ -2.96961248e+01]
 [  1.84953311e+00]
 [ -2.66343614e+00]
 [  2.74252258e+01]
 [ -3.50918199e+01]
 [  4.81915764e+01]
 [  3.09938313e+01]
 [  4.20211950e+01]
 [ -1.52110164e-01]
 [ -1.19311817e+00]
 [ -5.95431385e+00]
 [  8.91876180e-02]
 [  4.88557355e-01]
 [  4.07708203e+00]
 [  1.64692472e+00]
 [  2.43061349e+01]
 [  2.38542382e+01]
 [  1.58316579e+01]
 [ -1.93917342e-01]
 [  1.58016633e-02]
 [ -2.19327442e+00]
 [  1.54520277e-02]
 [ -2.99283113e-01]
 [  2.41070204e+01]]


/home/yann/Documents/higgs-boson/scripts/implementations.py:162: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))
/home/yann/Documents/higgs-boson/scripts/implementations.py:169: RuntimeWarning: divide by zero encountered in log
  return np.squeeze(-(y.T @ np.log(sigmoid(tx @ w)) + (1 - y).T @ np.log(1 - sigmoid(tx @ w))))


Current iteration=100, loss=nan
weights size:66112.34740211401
[[ -2.09662345e+02]
 [ -3.03219982e+00]
 [ -9.93658872e+01]
 [ -6.28932992e+00]
 [  3.51664070e+01]
 [  1.79666491e+01]
 [  1.99764370e+01]
 [ -1.37829149e+01]
 [  2.89831996e+01]
 [ -7.60888265e+00]
 [  1.81677208e+01]
 [ -4.69331471e+01]
 [  5.85425606e+01]
 [  2.40822676e+01]
 [  4.33150653e+01]
 [ -3.50628633e-01]
 [ -1.61805448e+00]
 [ -4.71300055e+00]
 [  1.38594179e-01]
 [  2.72893867e-01]
 [ -7.52555340e+00]
 [  1.44873780e+00]
 [  1.44624890e+01]
 [  1.36932104e+01]
 [  7.83425723e+00]
 [ -3.21833816e-01]
 [  3.08086049e-01]
 [ -8.25927112e+00]
 [  1.26660846e-01]
 [ -1.68574640e-01]
 [  1.26033705e+01]]
Current iteration=200, loss=nan
weights size:66112.34740211401
[[ -2.09662345e+02]
 [ -3.03219982e+00]
 [ -9.93658872e+01]
 [ -6.28932992e+00]
 [  3.51664070e+01]
 [  1.79666491e+01]
 [  1.99764370e+01]
 [ -1.37829149e+01]
 [  2.89831996e+01]
 [ -7.60888265e+00]
 [  1.81677208e+01]
 [ -4.69331471e+01]
 [  5.8542560

Current iteration=100, loss=nan
weights size:66180.21047446624
[[ -2.09787720e+02]
 [ -3.33989064e+00]
 [ -9.89598343e+01]
 [ -5.80466991e+00]
 [  3.51939301e+01]
 [  1.82688023e+01]
 [  1.98619722e+01]
 [ -1.40863170e+01]
 [  2.94995475e+01]
 [ -7.74021586e+00]
 [  1.85778891e+01]
 [ -4.65138378e+01]
 [  5.89577456e+01]
 [  2.41748424e+01]
 [  4.33549095e+01]
 [ -7.95037683e-01]
 [ -9.25965558e-01]
 [ -4.07877043e+00]
 [  2.27074517e-01]
 [  2.06125534e-01]
 [ -7.46520985e+00]
 [  7.94923569e-01]
 [  1.47103032e+01]
 [  1.42017054e+01]
 [  7.62863442e+00]
 [  2.39770051e-01]
 [  2.40156401e-01]
 [ -8.28073639e+00]
 [  4.31480193e-01]
 [ -4.26910576e-01]
 [  1.29356828e+01]]
Current iteration=200, loss=nan
weights size:66180.21047446625
[[ -2.09787720e+02]
 [ -3.33989064e+00]
 [ -9.89598343e+01]
 [ -5.80466991e+00]
 [  3.51939301e+01]
 [  1.82688023e+01]
 [  1.98619722e+01]
 [ -1.40863170e+01]
 [  2.94995475e+01]
 [ -7.74021586e+00]
 [  1.85778891e+01]
 [ -4.65138378e+01]
 [  5.8957745

(nan, nan, 0.71196400000000004, 0.71179599999999998)

In [99]:
# Create submission
y_prediction_test = predict_labels(weights, tx_test)
create_csv_submission(ids_test, y_prediction_test, "submission.csv")
